Using the optimum model found in sentimentAnalysisSVMExperimentation.ipynb I will now carry out sentiment analysis on twitter data using linearSVC trained on sentiment140 dataset

In [2]:
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy
spacy.load('en_core_web_sm')

import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

import numpy as np

import re

import contractions
from textacy.preprocessing import normalize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olija\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
preprocessedSentiment140Df = pd.read_csv("sentiment140PreprocessingExperiment.csv")

In [7]:
preprocessedSentiment140Df.head()

,sentimentScore,tweet,label,initialPreprocessTweet,initialPreprocessNoPunct,emojisReplaced,noStops,lemmatized,noShortWords,translateAbbreviations,...,preprocessorA,preprocessorB,preprocessorC,preprocessorD,preprocessorE,preprocessorF,preprocessorG,preprocessorH,preprocessorI,preprocessorJ
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative,"- awww, that's a bummer. you shoulda got david...",awww thats a bummer you shoulda got david carr...,awww thats a bummer you shoulda got david carr...,awww thats bummer shoulda got david carr third...,awww that s a bummer you shoulda get david car...,awww thats bummer shoulda david carr third,awww thats a bummer you shoulda got david carr...,...,aww that is a bummer you shoulda got david car...,awww that is a bummer you shoulda got david ca...,aww that is a bummer you shoulda got david car...,aww that is a bummer you shoulda got david car...,awww thats a bummer you shoulda got david carr...,aww thats a bummer you shoulda got david carr ...,aww thats a bummer you shoulda got david carr ...,awww that is a bummer you shoulda got david ca...,awww that is a bummer you shoulda got david ca...,aww that is a bummer you shoulda got david car...
1,0,is upset that he can't update his Facebook by ...,negative,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his facebook by t...,upset cant update facebook texting might cry r...,be upset that he can not update his facebook b...,upset that cant update facebook texting might ...,is upset that he cant update his facebook by t...,...,is upset that he cannot update his facebook by...,is upset that he cannot update his facebook by...,is upset that he cannot update his facebook by...,is upset that he cannot update his facebook by...,is upset that he cant update his facebook by t...,is upset that he cant update his facebook by t...,is upset that he cant update his facebook by t...,is upset that he cannot update his facebook by...,is upset that he cannot update his facebook by...,is upset that he cannot update his facebook by...
2,0,@Kenichan I dived many times for the ball. Man...,negative,i dived many times for the ball. managed to sa...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,dived many times ball managed save rest go bounds,I dive many time for the ball manage to save t...,dived many times ball managed save rest bounds,i dived many times for the ball managed to sav...,...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire,negative,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,whole body feels itchy like fire,my whole body feel itchy and like its on fire,whole body feels itchy like fire,my whole body feels itchy and like its on fire,...,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",negative,"no, it's not behaving at all. i'm mad. why am ...",no its not beh

In [ ]:
# This emoji's dicitonary was taken from:
# https://www.kaggle.com/code/stoicstatic/twitter-sentiment-analysis-for-beginners?scriptVersionId=68181216
# and adapted using emojis from:
# https://emojipedia.org/people/

emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat', '😀': 'grinning face', '😃': 'grinning face with big eyes', '😄': 'grinning face with smiling eyes', '😁': 'beaming face with smiling eyes', '😆': 'grinning squinting face', 
          '😅': 'grinning face with sweat', '🤣': 'rolling on the floor laughing', '😂': 'face with tears of joy', '🙂': 'slightly smiling face', '🙃': 'upside-down face', '😉': 'winking face', '😊': 'smiling face with smiling eyes', '😇': 'smiling face with halo', 
          '🥰': 'smiling face with hearts', '😍': 'smiling face with heart-eyes', '🤩': 'star-struck', '😘': 'face blowing a kiss', '😗': 'kissing face', '😚': 'kissing face with closed eyes', '😙': 'kissing face with smiling eyes', '🥲': 'smiling face with tear', 
          '😋': 'face savoring food', '😛': 'face with tongue', '😜': 'winking face with tongue', '🤪': 'zany face', '😝': 'squinting face with tongue', '🤑': 'money-mouth face', '🤗': 'smiling face with open hands', '🤭': 'face with hand over mouth', 
          '🤫': 'shushing face', '🤔': 'thinking face', '🤐': 'zipper-mouth face', '🤨': 'face with raised eyebrow', '😐': 'neutral face', '😑': 'expressionless face', '😶': 'face without mouth', '😏': 'smirking face', '😒': 'unamused face', '🙄': 'face with rolling eyes', 
          '😬': 'grimacing face', '🤥': 'lying face', '😌': 'relieved face', '😔': 'pensive face', '😪': 'sleepy face', '🤤': 'drooling face', '😴': 'sleeping face', '😷': 'face with medical mask', '🤒': 'face with thermometer', '🤕': 'face with head-bandage', 
          '🤢': 'nauseated face', '🤮': 'face vomiting', '🤧': 'sneezing face', '🥵': 'hot face', '🥶': 'cold face', '🥴': 'woozy face', '😵': 'face with crossed-out eyes', '🤯': 'exploding head', '🤠': 'cowboy hat face', '🥳': 'partying face', '🥸': 'disguised face', 
          '😎': 'smiling face with sunglasses', '🤓': 'nerd face', '🧐': 'face with monocle', '😕': 'confused face', '😟': 'worried face', '🙁': 'slightly frowning face', '😮': 'face with open mouth', '😯': 'hushed face', '😲': 'astonished face', '😳': 'flushed face', 
          '🥺': 'pleading face', '😦': 'frowning face with open mouth', '😧': 'anguished face', '😨': 'fearful face', '😰': 'anxious face with sweat', '😥': 'sad but relieved face', '😢': 'crying face', '😭': 'loudly crying face', '😱': 'face screaming in fear',
           '😖': 'confounded face', '😣': 'persevering face', '😞': 'disappointed face', '😓': 'downcast face with sweat', '😩': 'weary face', '😫': 'tired face', '🥱': 'yawning face', '😤': 'face with steam from nose', '😡': 'enraged face', '😠': 'angry face', 
           '🤬': 'face with symbols on mouth', '😈': 'smiling face with horns', '👿': 'angry face with horns', '💀': 'skull', '💩': 'pile of poo', '🤡': 'clown face', '👹': 'ogre', '👺': 'goblin', '👻': 'ghost', '👽': 'alien', '👾': 'alien monster', '🤖': 'robot', 
           '😺': 'grinning cat', '😸': 'grinning cat with smiling eyes', '😹': 'cat with tears of joy', '😻': 'smiling cat with heart-eyes', '😼': 'cat with wry smile', '😽': 'kissing cat', '🙀': 'weary cat', '😿': 'crying cat', '😾': 'pouting cat', '💋': 'kiss mark', 
           '👋': 'waving hand', '🤚': 'raised back of hand', '✋': 'raised hand', '🖖': 'vulcan salute', '👌': 'ok hand', '🤌': 'pinched fingers', '🤏': 'pinching hand', '🤞': 'crossed fingers', '🤟': 'love-you gesture', '🤘': 'sign of the horns', '🤙': 'call me hand', 
           '👈': 'backhand index pointing left', '👉': 'backhand index pointing right', '👆': 'backhand index pointing up', '🖕': 'middle finger', '👇': 'backhand index pointing down', '👍': 'thumbs up', '👎': 'thumbs down', '✊': 'raised fist', '👊': 'oncoming fist', 
           '🤛': 'left-facing fist', '🤜': 'right-facing fist', '👏': 'clapping hands', '🙌': 'raising hands', '👐': 'open hands', '🤲': 'palms up together', '🤝': 'handshake', '🙏': 'folded hands', '💅': 'nail polish', '🤳': 'selfie', '💪': 'flexed biceps', 
           '🦾': 'mechanical arm', '🦿': 'mechanical leg', '🦵': 'leg', '🦶': 'foot', '👂': 'ear', '🦻': 'ear with hearing aid', '👃': 'nose', '🧠': 'brain', '🫀': 'anatomical heart', '🫁': 'lungs', '🦷': 'tooth', '🦴': 'bone', '👀': 'eyes', '👅': 'tongue', '👄': 'mouth', 
           '👶': 'baby', '🧒': 'child', '👦': 'boy', '👧': 'girl', '🧑': 'person', '👱': 'person: blond hair', '👨': 'man', '🧔': 'person: beard', '👩': 'woman', '🧓': 'older person', '👴': 'old man', '👵': 'old woman', '🙍': 'person frowning', '🙎': 'person pouting', 
           '🙅': 'person gesturing no', '🙆': 'person gesturing ok', '💁': 'person tipping hand', '🙋': 'person raising hand', '🧏': 'deaf person', '🙇': 'person bowing', '🤦': 'person facepalming', '🤷': 'person shrugging', '👮': 'police officer', '💂': 'guard', '🥷': 'ninja', 
           '👷': 'construction worker', '🤴': 'prince', '👸': 'princess', '👳': 'person wearing turban', '👲': 'person with skullcap', '🧕': 'woman with headscarf', '🤵': 'person in tuxedo', '👰': 'person with veil', '🤰': 'pregnant woman', 
           '🤱': 'breast-feeding', '👼': 'baby angel', '🎅': 'santa claus', '🤶': 'mrs. claus', '🦸': 'superhero', '🦹': 'supervillain', '🧙': 'mage', '🧚': 'fairy', '🧛': 'vampire', '🧜': 'merperson', '🧝': 'elf', '🧞': 'genie', '🧟': 'zombie',
           '💆': 'person getting massage', '💇': 'person getting haircut', '🚶': 'person walking', '🧍': 'person standing', '🧎': 'person kneeling', '🏃': 'person running', '💃': 'woman dancing', '🕺': 'man dancing', '👯': 'people with bunny ears', 
           '🧖': 'person in steamy room', '🧘': 'person in lotus position', '👭': 'women holding hands', '👫': 'woman and man holding hands', '👬': 'men holding hands', '💏': 'kiss', '💑': 'couple with heart', '👪': 'family', '👤': 'bust in silhouette', 
           '👥': 'busts in silhouette', '🫂': 'people hugging', '👣': 'footprints', '🧳': 'luggage', '🌂': 'closed umbrella', '🎃': 'jack-o-lantern', '🧵': 'thread', '🧶': 'yarn', '👓': 'glasses', '🥽': 'goggles', '🥼': 'lab coat', '🦺': 'safety vest', 
           '👔': 'necktie', '👕': 't-shirt', '👖': 'jeans', '🧣': 'scarf', '🧤': 'gloves', '🧥': 'coat', '🧦': 'socks', '👗': 'dress', '👘': 'kimono', '🥻': 'sari', '🩱': 'one-piece swimsuit', '🩲': 'briefs', '🩳': 'shorts', '👙': 'bikini', '👚': 'woman’s clothes', 
           '👛': 'purse', '👜': 'handbag', '👝': 'clutch bag', '🎒': 'backpack', '🩴': 'thong sandal', '👞': 'man’s shoe', '👟': 'running shoe', '🥾': 'hiking boot', '🥿': 'flat shoe', '👠': 'high-heeled shoe', '👡': 'woman’s sandal', '🩰': 'ballet shoes',
            '👢': 'woman’s boot', '👑': 'crown', '👒': 'woman’s hat', '🎩': 'top hat', '🎓': 'graduation cap', '🧢': 'billed cap', '🪖': 'military helmet', '💄': 'lipstick', '💍': 'ring', '💼': 'briefcase', '🩸': 'drop of blood'}


htmlSymbols = {"&amp;": "and", "&quot;": '"', "&apos;": "'", "&cent;": "cent", "&pound;": "pound", "&yen;": "yen", "&euro;": "euro",
            "&copy;": "copyright", "&reg;": "registered trademark"}

# Abbreviation dictionary adapted from:
# https://www.socialmediatoday.com/social-networks/sarah-snow/2015-07-08/get-out-your-twittonary-twitter-abbreviations-you-must-know

abbreviationsDict = {'cc': 'carbon-copy', 'cx': 'correction', 'ct': 'cuttweet', 'dm': 'direct message', 'ht': 'heard through', 'mt': 'modified tweet', 
            'prt': 'please retweet', 'rt': 'retweet', 'sp': 'sponsored', 'em': 'email marketing', 'ezine': 'electronic magazine', 'fb': 'facebook', 
            'li': 'linkedin', 'seo': 'search engine optimization', 'sm': 'social media', 'smm': 'social media marketing', 'smo': 'social media optimization', 
            'sn': 'social network', 'sroi': 'social return on investment', 'ugc': 'user generated content', 'ux': 'user experience', 'yt': 'youtube', 
            'abt': 'about', 'afaik': 'as far as i know', 'ayfkmwts': 'are you fucking kidding me with this shit', 'b4': 'before', 'bfn': 'bye for now', 
            'bgd': 'background', 'bh': 'blockhead', 'br': 'best regards', 'btw': 'by the way', 'cd9': 'code 9', 'chk': 'check', 'cul8r': 'see you later', 
            'dam': 'donâ€™t annoy me', 'dd': 'dear daughter', 'df': 'dear fiance', 'ds': 'dear son', 'dyk': 'did you know', 'eml': 'email', 'ema': 'email address', 
            'f2f': 'face to face', 'ftf': 'face to face', 'ff': 'follow friday', 'ffs': "for fuck's sake", 'fml': 'fuck my life', 'fotd': 'find of the day', 
            'ftw': 'for the win', 'fubar': 'fucked up beyond all repair', 'fwiw': "for what it's worth", 'gmafb': 'give me a fucking break', 'gr8': 'great', 
            'gtfooh': 'get the fuck out of here', 'gts': 'guess the song', 'hagn': 'have a good night', 'hand': 'have a nice day', 'hotd': 'headline of the day', 
            'hth': 'hope that helps', 'ic': 'i see', 'icymi': 'in case you missed it', 'idk': "i don't know", 'iirc': 'if i remember correctly', 
            'imho': 'in my humble opinion', 'irl': 'in real life', 'iwsn': 'i want sex now', 'jk': 'joke', 'js': 'just saying', 'jsyk': 'just so you know', 
            'jv': 'joint venture', 'kk': 'okay', 'kyso': 'knock your socks off', 'lbs': 'laughing but serious', 'lhh': 'laugh hella hard', 'lmao': 'laughing my ass off', 
            'lmk': 'let me know', 'lo': 'little one', 'lol': 'laugh out loud', 'mm': 'music monday', 'mirl': 'meet in real life', 'mrjn': 'marijuana', 
            'msm': 'main stream media', 'mtf': 'more to follow', 'nbd': 'no big deal', 'nct': 'nobody cares though', 'nfi': 'no further information or not fucking interested', 
            'nfw': 'no fucking way', 'njoy': 'enjoy', 'nsfw': 'not safe for work', 'nts': 'note to self', 'oh': 'overheard', 'omfg': 'oh my fucking god', 
            'omg': 'oh my god', 'oomf': 'one of my friends', 'orly': 'oh really', 'plmk': 'please let me know', 'pnp': 'party and play', 'poidh': "pictures or it didn't happen", 
            'qotd': 'quote of the day', 're': 'in reply to', 'rlrt': 'real life retweet', 'rtfm': 'read the fucking manual', 'rtq': 'read the question', 'sfw': 'safe for work', 
            'smdh': 'shaking my damn head', 'smh': 'shaking my head', 'snafu': 'situation normal all fucked up', 'srs': 'serious', 'stfu': 'shut the fuck up', 
            'stfw': 'search the fucking web', 'tftf': 'thanks for the follow', 'tftt': 'thanks for this tweet', 'tj': 'tweetjack', 'tl': 'timeline', 'tldr': "too long didn't read", 
            'tmb': 'tweet me back', 'tt': 'trending topic', 'ty': 'thank you', 'tyia': 'thank you in advance', 'tyt': 'take your time', 'tyvm': 'thank you very much', 
            'w': 'with', 'wtv': 'whatever', 'ygtr': 'you got that right', 'ykwim': 'you know what i mean', 'ykyat': "you know you're addicted to", 
            'ymmv': 'your mileage may vary', 'yolo': 'you only live once', 'yoyo': "you're on your own", 'yw': "you're welcome", 'brb' : 'be right back', 'gtg' : 'got to go'}


stops = stopwords.words('english')

def appendStopsNoApostrophes(wordList):
    newList = wordList
    for word in wordList:
        if word.find("'") != -1:
            newList.append(word.replace("'", ""))
            newList.remove(word)
    return newList

stops = appendStopsNoApostrophes(stops)

STOPWORDS = set(stops)

In [ ]:
def initialPreprocessTweet(tweet):
     # to lower case
    cleanTweet = tweet.lower()
    # remove user mentions
    cleanTweet = re.sub("(@[A-Za-z0-9_]+)","", cleanTweet)
    # remove urls
    cleanTweet = re.sub("https?://[A-Za-z0-9./]*","", cleanTweet)
    # replace common html symbols
    for key in htmlSymbols.keys():
        cleanTweet = cleanTweet.replace(key, htmlSymbols[key])
    #normalize all whitespace, remove leading and trailing whitespace
    cleanTweet = normalize.whitespace(cleanTweet)    
    return cleanTweet

def replaceEmojis(tweet):
    #replace emojis with text translation
    cleanTweet = tweet
    for key in emojis.keys():
         cleanTweet = cleanTweet.replace(key, emojis[key])
    return cleanTweet

# expand abbreviations / acronyms - translate common abbreviations to their translation
def translateAbbreviations(tweet):
    cleanTweetList = tweet.split()
    for key in abbreviationsDict.keys():
        cleanTweetList = [abbreviationsDict[key] if item == key else item for item in cleanTweetList]
    return " ".join(cleanTweetList)

# method to expand English language contractions, including negation
def expandContractions(tweet):
    return contractions.fix(tweet)

# method for repeated letter normalization = goooooooood --> good (no english word has more than two consecutive letters the same)
def normalizeRepeatedChars(tweet):
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    cleanTweet = tweet
    for c in alphabet:
        cleanTweet = normalize.repeating_chars(cleanTweet, chars= c, maxn=2)
    return cleanTweet

def removeNonAlphaChars(tweet):
    # remove non alpha characters
    cleanTweet = re.sub("[^a-zA-Z ]","", tweet)
    #normalize all whitespace
    return normalize.whitespace(cleanTweet)    


#This is the preprocessor informed by experimenting with various techniques. Only those techniques that improve or match performance have been included
def preprocessorA(tweet):
    preprocessedTweet = initialPreprocessTweet(tweet)
    preprocessedTweet = replaceEmojis(preprocessedTweet)
    preprocessedTweet = removeNonAlphaChars(preprocessedTweet)
    preprocessedTweet = normalizeRepeatedChars(preprocessedTweet)
    preprocessedTweet = expandContractions(preprocessedTweet)
    preprocessedTweet = translateAbbreviations(preprocessedTweet)
    return preprocessedTweet


Methods to evaluate model

In [ ]:
# source: https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/
# https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
def evaluateModel(modelName, svcModel, xTest, yTest):
    y_pred = svcModel.predict(xTest)
    print(classification_report(yTest, y_pred))
    cf_matrix = confusion_matrix(yTest, y_pred)
    categories = ['Negative','Positive']
    group_names = ['True Negative','False Positive', 'False Negative','True Positive']
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names, group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot = labels, cmap = 'Reds',fmt = '', xticklabels = categories, yticklabels = categories)
    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values" , fontdict = {'size':14}, labelpad = 10)
    plt.title (modelName + " Confusion Matrix", fontdict = {'size':18}, pad = 20)

# source: https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/
def showROCCurve(modelName, svcModel, xTest, yTest):
    Y_pred1 = svcModel.predict(xTest)
    fpr, tpr, thresholds = roc_curve(yTest, Y_pred1)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkred', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(modelName + ' ROC CURVE')
    plt.legend(loc="lower right")
    plt.show()

def trainAndTestSVC(columnString, testSize, randomState, vectoriser, modelName):
    X_train,X_test,Y_train, Y_test = train_test_split(preprocessedSentiment140Df[columnString].apply(lambda x: np.str_(x)), 
    preprocessedSentiment140Df['sentimentScore'], test_size=testSize, random_state=randomState)  
    # ngram_range required is tuple e.g. (1,1) or (1,2)
    # ngram_range 1,1 means only unigrams are considered, 1,2 unigrams and bigrams
    # Transforming Xtrain and Xtest data using TF-IDF
    vectoriser = vectoriser
    vectoriser.fit_transform(X_train)
    print('No. of feature_words: ', len(vectoriser.get_feature_names_out()))
    X_train = vectoriser.transform(X_train)
    X_test  = vectoriser.transform(X_test)
    print("X_train shape: ", X_train.shape)
    print("Y_train shape: ", Y_train.shape)
    print("X_test shape: ", X_test.shape)
    print("Y_test shape: ", Y_test.shape)
    # Training SVM (LinearSVC) model
    svcModel = LinearSVC()
    svcModel.fit(X_train, Y_train)
    # evaluating model
    evaluateModel(modelName, svcModel, X_test, Y_test)
    showROCCurve(modelName, svcModel, X_test, Y_test)


In [ ]:
X_train,X_test,Y_train, Y_test = train_test_split(preprocessedSentiment140Df["preprocessorA"].apply(lambda x: np.str_(x)), 
preprocessedSentiment140Df['sentimentScore'], test_size=0.2, random_state=42)  
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=100000)
vectoriser.fit_transform(X_train)
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)
linearSVCClassifier = LinearSVC(C=0.1, loss='squared_hinge')
linearSVCClassifier.fit(X_train, Y_train)
evaluateModel("linearSVC Classifier", linearSVCClassifier, X_test, Y_test)
showROCCurve("linearSVC Classifier", linearSVCClassifier, X_test, Y_test)